In [ ]:
import mlflow
mlflow.start_run(run_name="analysis1", nested=True)

In [8]:
num_samples = 3
interval = 33.0e-3
num_frames = 100
min_sigma = 1
max_sigma = 4
threshold = 50.0
overlap = 0.5

In [ ]:
from mlflow import log_metric, log_param, log_artifacts
log_param("num_samples", num_samples)
log_param("interval", interval)
log_param("num_frames", num_frames)
log_param("min_sigma", min_sigma)
log_param("max_sigma", max_sigma)
log_param("threshold", threshold)
log_param("overlap", overlap)

In [9]:
nproc = 8

In [10]:
import numpy
timepoints = numpy.linspace(0, interval * num_frames, num_frames + 1)

In [11]:
import pathlib
inputpath = pathlib.Path("./artifacts")
artifacts = pathlib.Path("./artifacts")
artifacts.mkdir(parents=True, exist_ok=True)

In [12]:
import scopyon

In [13]:
import warnings
warnings.simplefilter('ignore', RuntimeWarning)

for i in range(num_samples):
    imgs = [scopyon.Image(data) for data in numpy.load(inputpath / f"images{i:03d}.npy")]
    spots = [
        scopyon.analysis.spot_detection(
            img.as_array(), processes=nproc,
            min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold, overlap=overlap)
        for img in imgs]

    spots_ = []
    for t, data in zip(timepoints, spots):
        spots_.extend(([t] + list(row) for row in data))
    spots_ = numpy.array(spots_)
    numpy.save(artifacts / f"spots{i:03d}.npy", spots_)
    
    print("{} spots are detected in {} frames.".format(len(spots_), len(imgs)))

warnings.resetwarnings()

19194 spots are detected in 150 frames.
18663 spots are detected in 150 frames.
18457 spots are detected in 150 frames.


In [14]:
!ls ./artifacts

analysis1.ipynb  images011.npy	images025.npy  inputs009.npy  inputs023.npy
analysis2.ipynb  images012.npy	images026.npy  inputs010.npy  inputs024.npy
config.yaml	 images013.npy	images027.npy  inputs011.npy  inputs025.npy
images000.npy	 images014.npy	images028.npy  inputs012.npy  inputs026.npy
images001.npy	 images015.npy	images029.npy  inputs013.npy  inputs027.npy
images002.npy	 images016.npy	inputs000.npy  inputs014.npy  inputs028.npy
images003.npy	 images017.npy	inputs001.npy  inputs015.npy  inputs029.npy
images004.npy	 images018.npy	inputs002.npy  inputs016.npy  spots000.npy
images005.npy	 images019.npy	inputs003.npy  inputs017.npy  spots001.npy
images006.npy	 images020.npy	inputs004.npy  inputs018.npy  spots002.npy
images007.npy	 images021.npy	inputs005.npy  inputs019.npy
images008.npy	 images022.npy	inputs006.npy  inputs020.npy
images009.npy	 images023.npy	inputs007.npy  inputs021.npy
images010.npy	 images024.npy	inputs008.npy  inputs022.npy


In [ ]:
log_artifacts("./artifacts")
mlflow.end_run()